### LOADING DATA AND CLEANING

In [43]:
from dataloader import create_cursor
import pandas as pd

In [45]:
con, cursor = create_cursor()

In [46]:
cursor.execute("""
            SELECT 
                Country,
                EdLevel,
                YearsCodePro,
                Employment,
                Salary 
            FROM 
               survey
        """)
df = pd.DataFrame(cursor.fetchall(), columns=[
    'Country',
    'EdLevel',
    'YearsCodePro','Employment','Salary'])


In [47]:
df

,Country,EdLevel,YearsCodePro,Employment,Salary
0,United States of America,"Bachelor’s degree (B.A., B.S., B.Eng., etc.)",9,"Employed, full-time",285000.0
1,United States of America,"Bachelor’s degree (B.A., B.S., B.Eng., etc.)",23,"Employed, full-time",250000.0
2,United States of America,"Bachelor’s degree (B.A., B.S., B.Eng., etc.)",7,"Employed, full-time",156000.0
3,Philippines,"Bachelor’s degree (B.A., B.S., B.Eng., etc.)",4,"Employed, full-time;Independent contractor, fr...",23456.0
4,United Kingdom of Great Britain and Northern I...,Some college/university study without earning ...,21,"Employed, full-time",96828.0
...,...,...,...,...,...
46554,Brazil,"Master’s degree (M.A., M.S., M.Eng., MBA, etc.)",24,"Employed, full-time",50719.0
46555,Armenia,Some college/university study without earning ...,2,"Employed, full-time;Independent contractor, fr...",16917.0
46556,India,"Bachelor’s degree (B.A., B.S., B.Eng., etc.)",2,"Employed, full-time",15752.0
46557,France,"Master’s degree (M.A., M.S., M.Eng., MBA, etc.)",9,"Employed, full-time",64254.0


In [48]:
df = df[df['Salary'].notnull()]

In [49]:
df = df.dropna()

In [50]:
df = df[df['Employment'] == 'Employed, full-time']
df = df.drop('Employment', axis=1)

In [51]:
def shorten_categories(categories, cutoff):
    cat_map = {}
    for c in range(len(categories)):
        if categories.values[c] >= cutoff:
            cat_map[categories.index[c]] = categories.index[c]
        else:
            cat_map[categories.index[c]] = 'Other'
    return cat_map


In [52]:
country_map = shorten_categories(df.Country.value_counts(), 400)
df['Country'] = df['Country'].map(country_map)

In [53]:
df = df[df['Salary'] <= 250_000]
df = df[df['Salary'] >= 10_000]
df = df[df['Salary'] != 'Other']


In [54]:
def clean_expr(x):
    if x == 'More than 50 years':
        return 50
    elif x == 'Less than 1 year':
        return 0.5
    return float(x)

df['YearsCodePro'] = df['YearsCodePro'].apply(clean_expr)

In [55]:
def clean_edu(x):
    if 'Bachelor’s degree' in x:
        return "Bachelor's degree"
    elif  'Master’s degree' in x:
        return "Master's degree"
    elif 'Professional degree' in x or 'Other doctoral' in x:
        return 'Post grad'
    return 'Less than a Bachelors'

df['EdLevel'] = df['EdLevel'].apply(clean_edu)

#### LABELING

In [56]:
from sklearn.preprocessing import LabelEncoder
le_education = LabelEncoder()
df['EdLevel'] = le_education.fit_transform(df['EdLevel'])
df['EdLevel'].unique()

array([0, 1, 2, 3])

In [57]:
le_country = LabelEncoder()
df['Country'] = le_country.fit_transform(df['Country'])

In [58]:
x = df.drop('Salary', axis=1)
y = df['Salary']

#### MODELS

In [59]:
from sklearn.linear_model import LinearRegression
linear_reg = LinearRegression()
linear_reg.fit(x, y.values)

LinearRegression()

In [60]:
y_pred = linear_reg.predict(x)

In [61]:
y_pred

array([137469.73003383, 107871.81652107, 128285.4796561 , ...,
        57620.29219559,  59600.35698015,  85692.6510228 ])

In [62]:
from sklearn.metrics import mean_squared_error, mean_absolute_error
import numpy as np 

error = np.sqrt(mean_squared_error(y, y_pred))

In [63]:
error

46626.50300917332

In [64]:
from sklearn.tree import DecisionTreeRegressor

dec_tree_reg = DecisionTreeRegressor(random_state=0)
dec_tree_reg.fit(x, y.values)

DecisionTreeRegressor(random_state=0)

In [65]:
y_pred = dec_tree_reg.predict(x)

In [66]:
error = np.sqrt(mean_squared_error(y, y_pred))

In [67]:
error

33308.46543178833

In [68]:
from sklearn.ensemble import RandomForestRegressor
random_forest_reg = RandomForestRegressor(random_state=0)
random_forest_reg.fit(x, y.values)

RandomForestRegressor(random_state=0)

In [69]:
y_pred = random_forest_reg.predict(x)

In [70]:
error = np.sqrt(mean_squared_error(y, y_pred))


In [71]:
error

33365.480241250705

In [72]:
from sklearn.model_selection import GridSearchCV

max_depth = [None, 2,4,6,8,10,12]
parameters = {'max_depth': max_depth}

regressor = DecisionTreeRegressor(random_state=0)
gs = GridSearchCV(regressor, parameters, scoring='neg_mean_squared_error')
gs.fit(x, y.values)

GridSearchCV(estimator=DecisionTreeRegressor(random_state=0),
             param_grid={'max_depth': [None, 2, 4, 6, 8, 10, 12]},
             scoring='neg_mean_squared_error')

In [73]:
regressor = gs.best_estimator_

regressor.fit(x, y.values)
y_pred = regressor.predict(x)
error = np.sqrt(mean_absolute_error(y, y_pred))



In [74]:
error

157.9712707381082

In [75]:
x

,Country,EdLevel,YearsCodePro
1,17,0,23.0
2,17,0,7.0
4,16,1,21.0
5,17,1,3.0
6,17,0,3.0
...,...,...,...
46553,11,3,5.0
46554,1,2,24.0
46556,6,0,2.0
46557,4,2,9.0


In [76]:
x = np.array([['United States of America', "Master's degree", 15]])

In [77]:
x

array([['United States of America', "Master's degree", '15']],
      dtype='<U24')

In [78]:
x[:, 0] = le_country.transform(x[:,0])
x[:, 1] = le_education.transform(x[:,1])
x = x.astype(float)
x

array([[17.,  2., 15.]])

In [79]:
y_pred = regressor.predict(x)
y_pred

/Users/radi/.pyenv/versions/3.11.2/lib/python3.11/site-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but DecisionTreeRegressor was fitted with feature names
  warnings.warn(


array([171952.90990991])

#### STORE MODEL

In [80]:
import pickle

In [81]:
data = {
    'model':regressor,
    'le_country': le_country,
    'le_education': le_education
}
with open('./data/steps.pkl', 'wb') as file:
    pickle.dump(data, file)

In [82]:
with open('./data/steps.pkl', 'rb') as file:
    data = pickle.load(file)

regressor_loades = data['model']
le_country = data['le_country']
le_education = data['le_education']


In [83]:
y_pred = regressor_loades.predict(x)
y_pred

/Users/radi/.pyenv/versions/3.11.2/lib/python3.11/site-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but DecisionTreeRegressor was fitted with feature names
  warnings.warn(


array([171952.90990991])